#Extended Kalman Filter Example

###Introduction

This notebook is designed to demonstrate how to use the StateSpace.jl package to execute the Extended Kalman filter for a non-linear State Space model. The example that has been used here closely follows Markus Gesmann's
Mages' blog titled [Extended Kalman filter example in R](http://www.magesblog.com/2015/01/extended-kalman-filter-example-in-r.html) which in turn
was based on a blog post titled [Fun with (Extended Kalman) Filters](https://idontgetoutmuch.wordpress.com/2014/09/09/fun-with-extended-kalman-filters-4/)
by Dominic Steinitz.

For those of you that do not need/want the explanation of the model and the code, you can skip right to the end of this notebook where the entire section of code required to run this example is given.

###The Problem
The problem that we will consider here is that of predicting the true value of a population (say of bacteria, but it could be anything) from some noisy measurements of the population. We also would like to predict the growth rate of the population, however we are faced with the problem that we do not observe the growth rate

#####Process  Model
It is assumed that the population evolves according to the logistic growth model. This is written mathematically as an ordinary differential equation:

$$\frac{\mathrm{d}p}{\mathrm{d}t} = rp\left(1 - \frac{p}{k} \right),$$

where $p$ is the population number, $t$ is the time, and $k$ is the carrying capacity.   
The solution to the above differetial equation is given by

$$p(t) = \frac{kp_0\exp(rt)}{k+p_0[\exp(rt) - 1]},$$

where $p_0$ is the initial population.   
This is almost in the form that we need. The only problem is that it needs to be formulated in a discrete manner whereby the population at the current time, $p_n$, is written in terms of the population and growth rate at the previous state, $p_{n-1}$ and $r_{n-1}$ respectively. This can be done like so:

$$p_n = \frac{kp_{n-1}\exp(r_{n-1}\Delta t)}{k+p_{n-1}[\exp(r_{n-1}\Delta t) - 1]},$$

where $\Delta t$ is the change in time from one measurement of the population to the next. This is our transistion (process) equation for the population.   

As mentioned previously, we also want to know the growth rate, and hence it is both the population number and the growth rate value together that define the state of our system. Hence we need to also define how the growth rate evolves. Here we will simply assume that the growth rate stays constant. Hence the transition equation for the growth rate will be written as:

$$r_n = r_{n-1}.$$

We will assume that these processes experience some noise. This will be in the form of zero mean Gaussian noise. We'll set the variance at 0.001 for both of these processes and hence we can write the full process equations as:

$$p_n = \frac{kp_{n-1}\exp(r_{n-1}\Delta t)}{k+p_{n-1}[\exp(r_{n-1}\Delta t) - 1]} + V_n \qquad V_n \sim \mathcal{N}(0, 0.001)$$

$$r_n = r_{t-n} + W_n \qquad W_n \sim \mathcal{N}(0, 0.001)$$

#####Observation Model
We assume that we observe the population directly but there is some zero mean Gaussian noise with variance = 25. Hence our observation model is:

$$y_n = p_n + Q_n \qquad Q_n \sim \mathcal{N}(0, 25),$$

where $y_n$ is the observation at the current time step.   

We do not observe the growth rate :(.

###Setting up the problem
First we'll import the required modules

In [1]:
using StateSpace
using Distributions
using Gadfly
using DataFrames
using Colors

#####Generate noisy observations
First thing we'll do is create the logistic function required to describe the evolution of the population

In [13]:
function logisticGrowth(r, p, k, t)
    k * p * exp(r*t) / (k + p * (exp(r*t) - 1))
end
logisticGrowth(state) = logisticGrowth(state[1], state[2], k, Δt)

logisticGrowth (generic function with 2 methods)

The second definition of the logistic function will come in useful when we are defining the process function.

Let's set the values of the parameters that we're going to use for the logistic function

In [14]:
r = 0.2
k = 100.0
p0 = 0.1 * k
Δt = 0.1

0.1

Now we'll set the variance of the observations (measurement noise)

In [15]:
measurement_noise_variance = 25.0

25.0

We can now define the number of observations and generate the true and observed population values. 

In [16]:
numObs = 100
true_values = Vector{Float64}(numObs)
population_measurements = Vector{Float64}(numObs)
for i in 1:numObs
    true_values[i] = logisticGrowth(r, p0, k, i*Δt)
    population_measurements[i] = true_values[i] + randn() * sqrt(measurement_noise_variance)
end

We don't observe the growth rate so we'll set these values to zero


In [17]:
growth_rate_measurements = zeros(numObs);

Finally we need to concatenate both population and growth rate arrays and ensure that have the correct dimensions

In [18]:
measurements = [growth_rate_measurements population_measurements]'

2x100 Array{Float64,2}:
  0.0        0.0      0.0     0.0      …   0.0      0.0      0.0      0.0   
 -0.775583  12.5597  11.2191  2.38745     52.0037  45.1471  45.9575  43.1948

#####Define Kalman Filter Parameters
We now need to define the process and observation model according to the model described earlier.

In [19]:
function process_fcn(state)
    predict_growth_rate = state[1]
    predict_population = logisticGrowth(state[1], state[2], k, Δt)
    new_state = [predict_growth_rate, predict_population]
    return new_state
end
process_noise_mat = diagm([0.001, 0.001])

function observation_fcn(state)
    growth_rate_observation = 0.0
    population_observation = 1.0 * state[2]
    observation = [growth_rate_observation, population_observation]
    return observation
end
observation_noise_mat = diagm([1.0, measurement_noise_variance])

2x2 Array{Float64,2}:
 1.0   0.0
 0.0  25.0

some couple of things to note about the above code:   
-The process of the growth and population are assumed uncorrelated, hence the off **diagonal terms are zero**. This is why the process noise matrix (and the observation noise matrix) are created as diagonal matrices.   
-Although the growth rate is not observed, we've still given the observation noise for the growth rate a **non-zero** value. It could be any non-zero value, I've just picked 1.0 (It shouldn't change the overall results). The problem with setting this value to zero is that you end up getting a singularity and hence the matrix algebra doesn't work and it will crash. So it must be a non zero value.   

We can now create an instance of the Nonlinear State Space Model

In [20]:
nonLinSSM = NonlinearGaussianSSM(process_fcn, process_noise_mat, observation_fcn, observation_noise_mat)

StateSpace.NonlinearGaussianSSM{Float64}(process_fcn,j,2x2 Array{Float64,2}:
 0.001  0.0  
 0.0    0.001,observation_fcn,j,2x2 Array{Float64,2}:
 1.0   0.0
 0.0  25.0)

#####Initial Guess
Now we can create an initial guess for the growth rate and the initial population along with the corresponding covariance matrix. 

In [21]:
initial_guess = MvNormal([0.5, 10], diagm([1.0,20.0]))

FullNormal(
dim: 2
μ: [0.5,10.0]
Σ: 2x2 Array{Float64,2}:
 1.0   0.0
 0.0  20.0
)


#####Perform Extended Kalman Filter Algorithm
Now we have all of the parameters:
1. noisy observations
2. process (transition) and observation (emission) model paramaters
3. initial guess of state   

We can use the Kalman Filter to predict the true underlying state (growth rate and population).

In [22]:
filtered_state = filter(nonLinSSM, measurements, initial_guess)

SmoothedState{Float64}


100 estimates of 2-D process from 2-D observations
Log-likelihood: -446.6972093371253


###Plot the results
Now we will plot the results. First we will look at the population estimates

First we extract the filtered population values along with their corresponding $2\sigma$ values (the give the area that we would expect the true value to lie with 95% confidence)

In [23]:
x_data = 1:numObs
population_array = Vector{Float64}(numObs+1)
confidence_array = Vector{Float64}(numObs+1)
population_array[1] = initial_guess.μ[2]
confidence_array[1] = 2*sqrt(initial_guess.Σ.mat[2,2])
for i in x_data
    current_state = filtered_state.state[i]
    population_array[i+1] = current_state.μ[2]
    confidence_array[i+1] = 2*sqrt(current_state.Σ.mat[2,2])
end

Next we will create a dataframe. This is simply so the syntax is simple for plotting the ribbon digram which will represent the state along with the confidence interval

In [25]:
df_fs = DataFrame(
    x = [0;x_data],
    y = population_array,
    ymin = population_array - confidence_array,
    ymax = population_array + confidence_array,
    f = "Filtered values"
    )

,x,y,ymin,ymax,f
1,0,10.0,1.0557280900008408,18.94427190999916,Filtered values
2,1,5.351396777457058,-1.5338036452913757,12.236597200205491,Filtered values
3,2,8.022672928101311,2.1405030139135413,13.904842842289082,Filtered values
4,3,9.454833778325888,3.935415055903305,14.97425250074847,Filtered values
5,4,7.955577771575199,2.4446685743490875,13.466486968801311,Filtered values
6,5,9.780412752282995,4.407073959551543,15.153751545014448,Filtered values
7,6,10.506390126267437,4.995507050630541,16.017273201904334,Filtered values
8,7,13.183649007357436,7.559573412939496,18.807724601775377,Filtered values
9,8,14.280404249743583,8.41111564533973,20.149692854147435,Filtered values
10,9,14.383015443144995,8.429969218033975,20.336061668256015,Filtered values


Next we will create separate colours for the observations and the true value. Here we will generate the default distinguishable colors used by Gadfly.

In [26]:
n = 3 #We will require 3 different colors
getColors = distinguishable_colors(n, Color[LCHab(70, 60, 240)],
                                   transform=c -> deuteranopic(c, 0.5),
                                   lchoices=Float64[65, 70, 75, 80],
                                   cchoices=Float64[0, 50, 60, 70],
                                   hchoices=linspace(0, 330, 24))

Finally we will plot the results of the filtered population estimates

In [27]:
population_state_plot = plot(
    layer(x=0:numObs, y=[p0;measurements[2,:]'], Geom.point, Theme(default_color=getColors[2])),
    layer(x=0:numObs, y=[p0;true_values], Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Measurement Number"), Guide.ylabel("Population"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Extended Kalman Filter Example")
    )
display(population_state_plot)

in call at C:\Users\jonathan\.julia\v0.4\Compose\src\svg.jl:226
in call at C:\Users\jonathan\.julia\v0.4\Compose\src\svg.jl:226
in call at C:\Users\jonathan\.julia\v0.4\Compose\src\svg.jl:226
in call at C:\Users\jonathan\.julia\v0.4\Compose\src\svg.jl:226
in call at C:\Users\jonathan\.julia\v0.4\Compose\src\svg.jl:226
in call at C:\Users\jonathan\.julia\v0.4\Compose\src\svg.jl:226
in call at C:\Users\jonathan\.julia\v0.4\Compose\src\svg.jl:226
in call at C:\Users\jonathan\.julia\v0.4\Compose\src\svg.jl:226
in compose at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:362
in compose at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:362
in compose at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:362
in compose at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:362
in compose at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:362
in compose at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:362
in compose at C:\Users\jonathan\.julia\v0.4\Compos

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Measurement Number 
 
 
 -150 
 -100 
 -50 
 0 
 50 
 100 
 150 
 200 
 250 
 -100 
 -95 
 -90 
 -85 
 -80 
 -75 
 -70 
 -65 
 -60 
 -55 
 -50 
 -45 
 -40 
 -35 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 60 
 65 
 70 
 75 
 80 
 85 
 90 
 95 
 100 
 105 
 110 
 115 
 120 
 125 
 130 
 135 
 140 
 145 
 150 
 155 
 160 
 165 
 170 
 175 
 180 
 185 
 190 
 195 
 200 
 -100 
 0 
 100 
 200 
 -100 
 -90 
 -80 
 -70 
 -60 
 -50 
 -40 
 -30 
 -20 
 -10 
 0 
 10 
 20 
 30 
 40 
 50 
 60 
 70 
 80 
 90 
 100 
 110 
 120 
 130 
 140 
 150 
 160 
 170 
 180 
 190 
 200 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M110.01,29.6 L 109.12 29.77 108.23 30.4 107.34 31.02 106.45 32.7 105.56 33.22 104.67 34.01 103.78 35.2 102.89 36.23 102 36.19 101.11 35.78 100.23 36.58 99.34 36.95 98.45 37.44 97.56 37.42 96.67 36.15 95.78 36.85 94.89 35.44 94 36.25 93.11 35.26 92.22 36.35 91.33 37.17 90.44 36.95 89.55 36.25 88.66 36.45 87.77 36.88 86.88 37.67 86 38.3 85.11 37.52 84.22 37.66 83.33 38.61 82.44 39.1 81.55 39.85 80.66 40.14 79.77 40.52 78.88 40.93 77.99 41.18 77.1 42.48 76.21 43.45 75.32 43.42 74.43 44.07 73.54 43.53 72.65 43.59 71.77 43.18 70.88 44.46 69.99 44.89 69.1 45.57 68.21 46.17 67.32 46.86 66.43 47.26 65.54 48.14 64.65 49.25 63.76 49.36 62.87 49.33 61.98 50.34 61.09 51.4 60.2 50.71 59.31 50.5 58.42 50.32 57.54 50.94 56.65 51.39 55.76 51.35 54.87 51.22 53.98 51.26 53.09 52.36 52.2 52.95 51.31 53.38 50.42 52.52 49.53 53.34 48.64 53.61 47.75 53.51 46.86 54.2 45.97 54.83 45.08 54.75 44.2 55.09 43.31 57.03 42.42 56.21 41.53 55.75 40.64 55.45 39.75 55.08 38.86 55.18 37.97 55.26 37.08 55.17 36.19 52.74 35.3 53.23 34.41 53.49 33.52 54.8 32.63 56.31 31.74 54.76 30.85 56.03 29.97 56.24 29.08 55.93 28.19 55.95 27.3 56.63 26.41 58.69 25.52 59.16 24.63 60.73 23.74 59.54 22.85 60.97 21.96 63.92 21.07 61.84 21.07 47.51 21.96 52.88 22.85 51.55 23.74 50.69 24.63 51.9 25.52 50.55 26.41 49.85 27.3 47.62 28.19 46.54 29.08 46.39 29.97 46.84 30.85 46.92 31.74 45.95 32.63 47.67 33.52 46.54 34.41 45.41 35.3 45.25 36.19 44.89 37.08 47.44 37.97 47.83 38.86 48.02 39.75 48.14 40.64 48.7 41.53 49.19 42.42 49.82 43.31 50.82 44.2 49.08 45.08 48.79 45.97 48.9 46.86 48.31 47.75 47.63 48.64 47.71 49.53 47.44 50.42 46.6 51.31 47.43 52.2 47.02 53.09 46.44 53.98 45.33 54.87 45.25 55.76 45.34 56.65 45.37 57.54 44.93 58.42 44.31 59.31 44.47 60.2 44.68 61.09 45.38 61.98 44.36 62.87 43.36 63.76 43.35 64.65 43.23 65.54 42.11 66.43 41.2 67.32 40.75 68.21 40.02 69.1 39.38 69.99 38.65 70.88 38.18 71.77 36.87 72.65 37.24 73.54 37.16 74.43 37.7 75.32 37.06 76.21 37.1 77.1 36.14 77.99 34.83 78.88 34.56 79.77 34.14 80.66 33.74 81.55 33.44 82.44 32.68 83.33 32.18 84.22 31.22 85.11 31.07 86 31.84 86.88 31.21 87.77 30.43 88.66 29.99 89.55 29.79 90.44 30.49 91.33 30.72 92.22 29.91 93.11 28.82 94 29.81 94.89 29 95.78 30.41 96.67 29.73 97.56 31 98.45 31.04 99.34 30.57 100.23 30.21 101.11 29.4 102 29.81 102.89 29.85 103.78 28.82 104.67 27.61 105.56 26.8 106.45 26.26 107.34 24.56 108.23 23.92 109.12 23.28 110.01 23.1 z"/>
 
 
 <path fill="none" d="M21.07,54.68 L 21.96 58.4 22.85 56.26 23.74 55.11 24.63 56.31 25.52 54.85 26.41 54.27 27.3 52.13 28.19 51.25 29.08 51.16 29.97 51.54 30.85 51.47 31.74 50.36 32.63 51.99 33.52 50.67 34.41 49.45 35.3 49.24 36.19 48.82 37.08 51.31 37.97 51.54 38.86 51.6 39.75 51.61 40.64 52.08 41.53 52.47 42.42 53.02 43.31 53.92 44.2 52.09 45.08 51.77 45.97 51.87 46.86

Void instead.
in drawpart at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:420
in drawpart at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:420
in drawpart at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:420
in drawpart at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:420
in drawpart at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:420
in drawpart at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:420
in drawpart at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:420
in drawpart at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:420
in drawpart at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:420
in realize_brute_force at C:\Users\jonathan\.julia\v0.4\Compose\src\table.jl:143
in realize_brute_force at C:\Users\jonathan\.julia\v0.4\Compose\src\table.jl:143
in realize_brute_force at C:\Users\jonathan\.julia\v0.4\Compose\src\table.jl:143
in realize_brute_force at C:\Users\jonathan\.julia\v0.4\Compose\src\table.jl:

We can use a similar approach to plot the filtered growth rate values

In [29]:
x_data = 1:numObs
growth_rate_array = Vector{Float64}(numObs+1)
confidence_array = Vector{Float64}(numObs+1)
growth_rate_array[1] = initial_guess.μ[1]
confidence_array[1] = initial_guess.Σ.mat[1,1]
for i in x_data
    current_state = filtered_state.state[i]
    growth_rate_array[i+1] = current_state.μ[1]
    confidence_array[i+1] = 2*sqrt(current_state.Σ.mat[1,1])
end
df_fs = DataFrame(
    x = [0;x_data],
    y = growth_rate_array,
    ymin = growth_rate_array - confidence_array,
    ymax = growth_rate_array + confidence_array,
    f = "Filtered values"
    )

growth_rate_state_plot = plot(
    layer(x=0:numObs, y=ones(numObs+1)*r, Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Measurement Number"), Guide.ylabel("Growth Rate"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Extended Kalman Filter Example")
    )
display(growth_rate_state_plot)

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in compose at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:334
 in render_prepared at C:\Users\jonathan\.julia\v0.4\Gadfly\src\Gadfly.jl:782
 in render at C:\Users\jonathan\.julia\v0.4\Gadfly\src\Gadfly.jl:721
 in writemime at C:\Users\jonathan\.julia\v0.4\Gadfly\src\Gadfly.jl:845
 in sprint at iostream.jl:211
 in stringmime at multimedia.jl:73
 in display at C:\Users\jonathan\.julia\v0.4\IJulia\src\inline.jl:19
 [inlined code] from multimedia.jl:151
 in display at C:\Users\jonathan\.julia\v0.4\Gadfly\src\Gadfly.jl:921
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at C:\Users\jonathan\.julia\v0.4\IJulia\src\execute_request.jl:177
 in eventloop at C:\Users\jonathan\.julia\v0.4\IJulia\src\IJulia.jl:144
 in anonymous at task.jl:447
while loading In[29], in expression starting on line 26
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in compose at C:\Users\jonathan\.julia\v0.4\Compo

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Measurement Number 
 
 
 -150 
 -100 
 -50 
 0 
 50 
 100 
 150 
 200 
 250 
 -100 
 -95 
 -90 
 -85 
 -80 
 -75 
 -70 
 -65 
 -60 
 -55 
 -50 
 -45 
 -40 
 -35 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 60 
 65 
 70 
 75 
 80 
 85 
 90 
 95 
 100 
 105 
 110 
 115 
 120 
 125 
 130 
 135 
 140 
 145 
 150 
 155 
 160 
 165 
 170 
 175 
 180 
 185 
 190 
 195 
 200 
 -100 
 0 
 100 
 200 
 -100 
 -90 
 -80 
 -70 
 -60 
 -50 
 -40 
 -30 
 -20 
 -10 
 0 
 10 
 20 
 30 
 40 
 50 
 60 
 70 
 80 
 90 
 100 
 110 
 120 
 130 
 140 
 150 
 160 
 170 
 180 
 190 
 200 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M110.01,53.18 L 109.1 52.99 108.2 53.07 107.29 53.15 106.38 53.92 105.48 54.02 104.57 54.3 103.67 54.9 102.76 55.45 101.85 55.37 100.95 55.03 100.04 55.46 99.13 55.64 98.23 55.93 97.32 55.93 96.42 55.1 95.51 55.45 94.6 54.47 93.7 54.82 92.79 54.04 91.89 54.51 90.98 54.86 90.07 54.58 89.17 53.95 88.26 53.82 87.36 53.84 86.45 54.09 85.54 54.26 84.64 53.52 83.73 53.29 82.83 53.58 81.92 53.59 81.01 53.79 80.11 53.69 79.2 53.65 78.29 53.61 77.39 53.46 76.48 54.02 75.58 54.43 74.67 54.19 73.76 54.4 72.86 53.79 71.95 53.54 71.05 52.9 70.14 53.4 69.23 53.36 68.33 53.5 67.42 53.6 66.52 53.8 65.61 53.8 64.7 54.17 63.8 54.78 62.89 54.69 61.98 54.48 61.08 55.06 60.17 55.76 59.27 55.16 58.36 54.86 57.45 54.54 56.55 54.83 55.64 55.03 54.74 54.85 53.83 54.57 52.92 54.39 52.02 55.13 51.11 55.52 50.21 55.82 49.3 54.97 48.39 55.58 47.49 55.75 46.58 55.58 45.68 56.16 44.77 56.76 43.86 56.72 42.96 57.1 42.05 59.11 41.14 58.53 40.24 58.27 39.33 58.15 38.43 57.93 37.52 58.19 36.61 58.47 35.71 58.58 34.8 55.77 33.9 56.18 32.99 56.31 32.08 58.11 31.18 60.63 30.27 58.21 29.37 60.53 28.46 61.15 27.55 60.44 26.65 59.91 25.74 60.88 24.83 67.15 23.93 68.89 23.02 74.26 22.12 70.3 21.21 72.32 20.3 75.02 19.4 59.48 19.4 33.84 20.3 24.19 21.21 22.17 22.12 21.61 23.02 28.17 23.93 25.78 24.83 27.72 25.74 25.45 26.65 28.84 27.55 33.49 28.46 37.71 29.37 39.91 30.27 39.86 31.18 44.22 32.08 43.22 32.99 42.71 33.9 43.71 34.8 44.28 35.71 47.93 36.61 48.49 37.52 48.76 38.43 48.94 39.33 49.53 40.24 49.96 41.14 50.47 42.05 51.25 42.96 49.4 43.86 49.16 44.77 49.33 45.68 48.84 46.58 48.36 47.49 48.63 48.39 48.56 49.3 48.03 50.21 48.97 51.11 48.75 52.02 48.42 52.92 47.75 53.83 47.99 54.74 48.33 55.64 48.56 56.55 48.41 57.45 48.16 58.36 48.52 59.27 48.85 60.17 49.49 61.08 48.82 61.98 48.25 62.89 48.49 63.8 48.6 64.7 48.01 65.61 47.65 66.52 47.68 67.42 47.52 68.33 47.44 69.23 47.33 70.14 47.41 71.05 46.94 71.95 47.62 72.86 47.9 73.76 48.54 74.67 48.35 75.58 48.61 76.48 48.22 77.39 47.68 78.29 47.84 79.2 47.9 80.11 47.96 81.01 48.07 81.92 47.89 82.83 47.9 83.73 47.62 84.64 47.87 85.54 48.62 86.45 48.46 87.36 48.22 88.26 48.22 89.17 48.36 90.07 49 90.98 49.28 91.89 48.93 92.79 48.47 93.7 49.26 94.6 48.91 95.51 49.89 96.42 49.54 97.32 50.37 98.23 50.38 99.13 50.09 100.04 49.9 100.95 49.48 101.85 49.82 102.76 49.89 103.67 49.34 104.57 48.75 105.48 48.47 106.38 48.37 107.29 47.62 108.2 47.55 109.1 47.48 110.01 47.68 z"/>
 
 
 <path fill="none" d="M19.4,46.66 L 20.3 49.61 21.21 47.25 22.12 45.96 23.02 51.22 23.93 47.33 24.83 47.43 25.74 43.16 26.65 44.37 27.55 46.96 28.46 49.43 29.37 50.22 30.27 49.04 31.18 52.42 32.08 50.66 32.99 49.51 33.9 49.94 34.8 50.03 35.71 53.26 36.61 53.48 37.52 53.48 38.43 53.44 39.33 53.84 40.24 54.11 41.14 54.5 42.05 55.18 42

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in realize_brute_force at C:\Users\jonathan\.julia\v0.4\Compose\src\table.jl:352
 in realize at C:\Users\jonathan\.julia\v0.4\Compose\src\table.jl:378
 in drawpart at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:447
 in draw at C:\Users\jonathan\.julia\v0.4\Compose\src\container.jl:404
 in writemime at C:\Users\jonathan\.julia\v0.4\Gadfly\src\Gadfly.jl:845
 in sprint at iostream.jl:211
 in stringmime at multimedia.jl:73
 in display at C:\Users\jonathan\.julia\v0.4\IJulia\src\inline.jl:19
 [inlined code] from multimedia.jl:151
 in display at C:\Users\jonathan\.julia\v0.4\Gadfly\src\Gadfly.jl:921
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at C:\Users\jonathan\.julia\v0.4\IJulia\src\execute_request.jl:177
 in eventloop at C:\Users\jonathan\.julia\v0.4\IJulia\src\IJulia.jl:144
 in anonymous at task.jl:447
while loading In[29], in expression starting on line 26
 in depwarn at deprecated.jl:73
 i

For those that just want the code, without the explanation, you have come to the right place. Here it is:

In [ ]:
using StateSpace
using Distributions
using Gadfly
using DataFrames
using Colors

r = 0.2 #r is the growth rate
k = 100.0 #k is the carrying capacity
p0 = 0.1 * k # p0 is the initial population
Δt = 0.1 # Δt is the change in time.

#Define the logistic growth function to set the observations
function logisticGrowth(r, p, k, t)
    k * p * exp(r*t) / (k + p * (exp(r*t) - 1))
end
logisticGrowth(state) = logisticGrowth(state[1], state[2], k, Δt)

#Generate noisy measurements
measurement_noise_variance = 25.0
numObs = 100
true_values = Vector{Float64}(numObs)
population_measurements = Vector{Float64}(numObs)
for i in 1:numObs
    true_values[i] = logisticGrowth(r, p0, k, i*Δt)
    population_measurements[i] = true_values[i] + randn() * sqrt(measurement_noise_variance)
end
growth_rate_measurements = zeros(numObs)
measurements = [growth_rate_measurements population_measurements]'

#Section: Describe Extended Kalman Filter parameters
function process_fcn(state)
    predict_growth_rate = state[1]
    predict_population = logisticGrowth(state)
    new_state = [predict_growth_rate, predict_population]
    return new_state
end
process_noise_mat = diagm([0.001, 0.001])

function observation_fcn(state)
    growth_rate_observation = 0.0
    population_observation = 1.0 * state[2]
    observation = [growth_rate_observation, population_observation]
    return observation
end
observation_noise_mat = diagm([1.0, measurement_noise_variance])

#Create instance of our EKF model
nonLinSSM = NonlinearGaussianSSM(process_fcn, process_noise_mat, observation_fcn, observation_noise_mat)

initial_guess = MvNormal([0.5, 10], diagm([1.0,20.0])) #Set initial guess of the state

filtered_state = filter(nonLinSSM, measurements, initial_guess) #Execute  the Extended Kalman Filter

#Plot Filtered results for population
x_data = 1:numObs
population_array = Vector{Float64}(numObs+1)
confidence_array = Vector{Float64}(numObs+1)
population_array[1] = initial_guess.μ[2]
confidence_array[1] = 2*sqrt(initial_guess.Σ.mat[2,2])
for i in x_data
    current_state = filtered_state.state[i]
    population_array[i+1] = current_state.μ[2]
    confidence_array[i+1] = 2*sqrt(current_state.Σ.mat[2,2])
end
df_fs = DataFrame(
    x = [0;x_data],
    y = population_array,
    ymin = population_array - confidence_array,
    ymax = population_array + confidence_array,
    f = "Filtered values"
    )

n = 3
getColors = distinguishable_colors(n, Color[LCHab(70, 60, 240)],
                                   transform=c -> deuteranopic(c, 0.5),
                                   lchoices=Float64[65, 70, 75, 80],
                                   cchoices=Float64[0, 50, 60, 70],
                                   hchoices=linspace(0, 330, 24))
population_state_plot = plot(
    layer(x=0:numObs, y=[p0;measurements[2,:]'], Geom.point, Theme(default_color=getColors[2])),
    layer(x=0:numObs, y=[p0;true_values], Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Measurement Number"), Guide.ylabel("Population"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Extended Kalman Filter Example")
    )
display(population_state_plot)

#Plot Filtered results for growth rate
x_data = 1:numObs
growth_rate_array = Vector{Float64}(numObs+1)
confidence_array = Vector{Float64}(numObs+1)
growth_rate_array[1] = initial_guess.μ[1]
confidence_array[1] = initial_guess.Σ.mat[1,1]
for i in x_data
    current_state = filtered_state.state[i]
    growth_rate_array[i+1] = current_state.μ[1]
    confidence_array[i+1] = 2*sqrt(current_state.Σ.mat[1,1])
end
df_fs = DataFrame(
    x = [0;x_data],
    y = growth_rate_array,
    ymin = growth_rate_array - confidence_array,
    ymax = growth_rate_array + confidence_array,
    f = "Filtered values"
    )

growth_rate_state_plot = plot(
    layer(x=0:numObs, y=ones(numObs+1)*r, Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Measurement Number"), Guide.ylabel("Growth Rate"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Extended Kalman Filter Example")
    )
display(growth_rate_state_plot)